In [2]:
import os
import numpy as np
import pydicom
import cv2
import matplotlib.pyplot as plt
import pandas as pd

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

In [3]:
def resize_image(image_path):
    dicom = pydicom.dcmread(image_path)

    # Convert the DCM image to a numpy array
    image_array = dicom.pixel_array

    # Resize the image using OpenCV
    resized_image = cv2.resize(image_array, (224, 224))

    return resized_image

In [4]:
# Specify the path to the main directory containing patient folders
main_directory = './small_train_images'

# Dictionary to store image data
image_data = {}

# Iterate over study_id folders
for study_id in os.listdir(main_directory):
    study_id_dir = os.listdir(f'./small_train_images/{study_id}')
    
    # Iterate over series folders for each patient
    for series_id in study_id_dir:
        series_id_dir = os.listdir(f'./small_train_images/{study_id}/{series_id}')
        # Initialize list to store image arrays for the series
        image_arrays = []
        
        # Iterate over DICOM files in the series folder
        for instance in series_id_dir:
            image_path = f'./small_train_images/{study_id}/{series_id}/{instance}'
            resized_image = resize_image(image_path)

            # Append resized_image array to the list
            image_arrays.append(resized_image)
        
        # Vertically stack DCM images
        stacked_images = np.vstack(image_arrays)
        # Store stacked images as a NumPy array
        np_array = np.array(stacked_images)
        print(series_id, np_array.shape)

        # Store image arrays in the dictionary with (study_id, series_id) tuple as key
        image_data[(study_id, series_id)] = np_array

142859125 (8064, 224)
2073726394 (10304, 224)
2399638375 (4256, 224)
3491739931 (4256, 224)
1224932122 (10080, 224)
2231042680 (4032, 224)
3543553307 (4032, 224)
1212326388 (3360, 224)
1638921810 (3360, 224)
3800798510 (6048, 224)
403244853 (6272, 224)
1539051863 (3808, 224)
2500166693 (6048, 224)
2677627096 (3808, 224)
3687121182 (12992, 224)
3753885158 (4032, 224)
434280813 (4032, 224)
1679014482 (10080, 224)
226564374 (3808, 224)
2528347280 (3808, 224)
307069509 (5600, 224)
1152175603 (5152, 224)
1676821058 (5152, 224)
2261718442 (6720, 224)
231278500 (8960, 224)
1379151387 (3360, 224)
1847558962 (4928, 224)
758801267 (3360, 224)
1054713880 (3360, 224)
2448190387 (9632, 224)
702807833 (3360, 224)
3201256954 (12096, 224)
3486248476 (3808, 224)
3666319702 (3808, 224)
132939515 (3808, 224)
1951927562 (5152, 224)
3219733239 (3808, 224)
1570286759 (3360, 224)
2406919186 (4704, 224)
481125819 (3360, 224)


In [5]:
# Check shapes
example_study_id = '4003253'
example_series_id = '702807833'
if (example_study_id, example_series_id) in image_data:
    image = image_data[(example_study_id, example_series_id)]
    print(f"Image shape:", image.shape)
else:
    print("No images found for the specified (study_id, series_id) tuple.")

Image shape: (3360, 224)


In [6]:
# Pad arrays

# Find the maximum shape among all numpy arrays
max_shape = max([np_array.shape for np_array in image_data.values()], key=lambda x: x[0])

for key in image_data:
    np_array = image_data[key]
    padding = max_shape[0] - np_array.shape[0]
    if padding > 0:
        padding_shape = ((0, padding), (0, 0))
        padded_np_array = np.pad(np_array, padding_shape, mode='constant', constant_values=0) # 0 is black (If I'm not mistaken)
        image_data[key] = padded_np_array

# Print the shapes of padded numpy arrays
for key in image_data:
    print(key, image_data[key].shape)

('10728036', '142859125') (12992, 224)
('10728036', '2073726394') (12992, 224)
('10728036', '2399638375') (12992, 224)
('10728036', '3491739931') (12992, 224)
('11340341', '1224932122') (12992, 224)
('11340341', '2231042680') (12992, 224)
('11340341', '3543553307') (12992, 224)
('11943292', '1212326388') (12992, 224)
('11943292', '1638921810') (12992, 224)
('11943292', '3800798510') (12992, 224)
('11943292', '403244853') (12992, 224)
('13317052', '1539051863') (12992, 224)
('13317052', '2500166693') (12992, 224)
('13317052', '2677627096') (12992, 224)
('22191399', '3687121182') (12992, 224)
('22191399', '3753885158') (12992, 224)
('22191399', '434280813') (12992, 224)
('26342422', '1679014482') (12992, 224)
('26342422', '226564374') (12992, 224)
('26342422', '2528347280') (12992, 224)
('26342422', '307069509') (12992, 224)
('29931867', '1152175603') (12992, 224)
('29931867', '1676821058') (12992, 224)
('29931867', '2261718442') (12992, 224)
('29931867', '231278500') (12992, 224)
('3373

In [9]:
# Make 3D arrays

# Group arrays by the first tuple values
grouped_arrays = {}
for key, value in image_data.items():
    if key[0] not in grouped_arrays:
        grouped_arrays[key[0]] = [value]
    else:
        grouped_arrays[key[0]].append(value)

# Stack arrays with the same first tuple values into a 3D array
stacked_arrays = {}
for key, values in grouped_arrays.items():
    stacked_arrays[key] = np.stack(values, axis=0)

stacked_tensors = {}
for key, value in stacked_arrays.items():
    stacked_tensors[key] = torch.from_numpy(value)

# Check the stacked arrays
for key, value in stacked_tensors.items():
    print(f"Stacked arrays for key {key}: {value.shape}")

Stacked arrays for key 10728036: torch.Size([4, 12992, 224])
Stacked arrays for key 11340341: torch.Size([3, 12992, 224])
Stacked arrays for key 11943292: torch.Size([4, 12992, 224])
Stacked arrays for key 13317052: torch.Size([3, 12992, 224])
Stacked arrays for key 22191399: torch.Size([3, 12992, 224])
Stacked arrays for key 26342422: torch.Size([4, 12992, 224])
Stacked arrays for key 29931867: torch.Size([4, 12992, 224])
Stacked arrays for key 33736057: torch.Size([3, 12992, 224])
Stacked arrays for key 4003253: torch.Size([3, 12992, 224])
Stacked arrays for key 4646740: torch.Size([3, 12992, 224])
Stacked arrays for key 7143189: torch.Size([3, 12992, 224])
Stacked arrays for key 8785691: torch.Size([3, 12992, 224])


In [10]:
# Determine the maximum shape
max_shape = tuple(max(arr.shape[i] for arr in stacked_tensors.values()) for i in range(3))

# Pad each tensor to the maximum shape
for key, tensor in stacked_tensors.items():
    pad_shape = [max_shape[i] - tensor.shape[i] for i in range(3)]
    stacked_tensors[key] = F.pad(tensor, (0, pad_shape[2], 0, pad_shape[1], 0, pad_shape[0]))

In [12]:
# Print the new shapes
for key, arr in stacked_tensors.items():
    print(f"Key: {key}, New Shape: {arr.shape}")

Key: 10728036, New Shape: torch.Size([4, 12992, 224])
Key: 11340341, New Shape: torch.Size([4, 12992, 224])
Key: 11943292, New Shape: torch.Size([4, 12992, 224])
Key: 13317052, New Shape: torch.Size([4, 12992, 224])
Key: 22191399, New Shape: torch.Size([4, 12992, 224])
Key: 26342422, New Shape: torch.Size([4, 12992, 224])
Key: 29931867, New Shape: torch.Size([4, 12992, 224])
Key: 33736057, New Shape: torch.Size([4, 12992, 224])
Key: 4003253, New Shape: torch.Size([4, 12992, 224])
Key: 4646740, New Shape: torch.Size([4, 12992, 224])
Key: 7143189, New Shape: torch.Size([4, 12992, 224])
Key: 8785691, New Shape: torch.Size([4, 12992, 224])


In [22]:
# Add targets
targets_df = pd.read_csv('./train.csv')

mapping = {
    'normal/mild': 1,
    'moderate': 2,
    'severe': 3
}
def standardize_and_map(column, mapping):
    return column.str.lower().str.strip().map(mapping)

# List of columns to apply the transformation
columns_to_transform = ['spinal_canal_stenosis_l1_l2',
       'spinal_canal_stenosis_l2_l3', 'spinal_canal_stenosis_l3_l4',
       'spinal_canal_stenosis_l4_l5', 'spinal_canal_stenosis_l5_s1',
       'left_neural_foraminal_narrowing_l1_l2',
       'left_neural_foraminal_narrowing_l2_l3',
       'left_neural_foraminal_narrowing_l3_l4',
       'left_neural_foraminal_narrowing_l4_l5',
       'left_neural_foraminal_narrowing_l5_s1',
       'right_neural_foraminal_narrowing_l1_l2',
       'right_neural_foraminal_narrowing_l2_l3',
       'right_neural_foraminal_narrowing_l3_l4',
       'right_neural_foraminal_narrowing_l4_l5',
       'right_neural_foraminal_narrowing_l5_s1',
       'left_subarticular_stenosis_l1_l2', 'left_subarticular_stenosis_l2_l3',
       'left_subarticular_stenosis_l3_l4', 'left_subarticular_stenosis_l4_l5',
       'left_subarticular_stenosis_l5_s1', 'right_subarticular_stenosis_l1_l2',
       'right_subarticular_stenosis_l2_l3',
       'right_subarticular_stenosis_l3_l4',
       'right_subarticular_stenosis_l4_l5',
       'right_subarticular_stenosis_l5_s1']


In [23]:
# Apply the function to the specified columns
for column in columns_to_transform:
    targets_df[column] = standardize_and_map(targets_df[column], mapping)

In [68]:
test_df = targets_df[targets_df['study_id'] == 4003253]
test_df

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0


In [75]:
# # Store targets in a dict as numpy arrays and rashape them

# targets_dict = {}

# for key, arr in stacked_tensors.items():
#     target = targets_df[targets_df['study_id'] == int(key)]
#     target_study_id = target.iloc[:, :1]
#     target_values_int = np.array(target.iloc[:, 1:]).flatten().astype(int) - 1
#     # target_values = target.iloc[:, 1:]

#     # Convert the target array to one-hot encoding with three classes
#     num_classes = 3
#     # one_hot_targets = np.eye(num_classes)[np.array(target_values).flatten() - 1]  # Subtracting 1 to make classes start from 0
#     one_hot_targets = np.eye(num_classes)[target_values_int]

#     # Reshape the one-hot encoded target array to match the desired output shape (25, 3)
#     reshaped_targets = one_hot_targets.reshape(-1, num_classes)

#     targets_dict[target_study_id.iloc[0, 0]] = reshaped_targets

In [78]:
# Store targets in a dict as torch tensors and rashape them

targets_tensors = {}

for key, arr in stacked_tensors.items():
    target = targets_df[targets_df['study_id'] == int(key)]
    target_study_id = target.iloc[:, :1]
    target_values_int = np.array(target.iloc[:, 1:]).flatten().astype(int) - 1

    # Convert the target array to one-hot encoding with three classes
    num_classes = 3
    one_hot_targets = np.eye(num_classes)[target_values_int]

    # Reshape the one-hot encoded target array to match the desired output shape (25, 3)
    reshaped_targets = one_hot_targets.reshape(-1, num_classes)

    # Convert the numpy array to a torch tensor
    tensor_targets = torch.from_numpy(reshaped_targets)

    targets_tensors[target_study_id.iloc[0, 0]] = tensor_targets

In [97]:
# Print the new shapes
for key, arr in targets_tensors.items():
    print(f"Key: {key}, New Shape: {arr.shape}")
    # if key == 4003253:
    #     print(arr)

Key: 10728036, New Shape: torch.Size([25, 3])
Key: 11340341, New Shape: torch.Size([25, 3])
Key: 11943292, New Shape: torch.Size([25, 3])
Key: 13317052, New Shape: torch.Size([25, 3])
Key: 22191399, New Shape: torch.Size([25, 3])
Key: 26342422, New Shape: torch.Size([25, 3])
Key: 29931867, New Shape: torch.Size([25, 3])
Key: 33736057, New Shape: torch.Size([25, 3])
Key: 4003253, New Shape: torch.Size([25, 3])
Key: 4646740, New Shape: torch.Size([25, 3])
Key: 7143189, New Shape: torch.Size([25, 3])
Key: 8785691, New Shape: torch.Size([25, 3])


In [108]:
for ((key, feature),(key2, targets)) in zip(stacked_tensors.items(), targets_tensors.items()):
    print(f"Key: {key}, New Shape: {feature.shape} | Key: {key2}, New Shape: {targets.shape}")

Key: 10728036, New Shape: torch.Size([4, 12992, 224]) | Key: 10728036, New Shape: torch.Size([25, 3])
Key: 11340341, New Shape: torch.Size([4, 12992, 224]) | Key: 11340341, New Shape: torch.Size([25, 3])
Key: 11943292, New Shape: torch.Size([4, 12992, 224]) | Key: 11943292, New Shape: torch.Size([25, 3])
Key: 13317052, New Shape: torch.Size([4, 12992, 224]) | Key: 13317052, New Shape: torch.Size([25, 3])
Key: 22191399, New Shape: torch.Size([4, 12992, 224]) | Key: 22191399, New Shape: torch.Size([25, 3])
Key: 26342422, New Shape: torch.Size([4, 12992, 224]) | Key: 26342422, New Shape: torch.Size([25, 3])
Key: 29931867, New Shape: torch.Size([4, 12992, 224]) | Key: 29931867, New Shape: torch.Size([25, 3])
Key: 33736057, New Shape: torch.Size([4, 12992, 224]) | Key: 33736057, New Shape: torch.Size([25, 3])
Key: 4003253, New Shape: torch.Size([4, 12992, 224]) | Key: 4003253, New Shape: torch.Size([25, 3])
Key: 4646740, New Shape: torch.Size([4, 12992, 224]) | Key: 4646740, New Shape: torc

In [109]:
# Convert all tensors to float32
feature_tensorstest = [tensor.float() for tensor in stacked_tensors.values()]
target_tensorstest = [tensor.float() for tensor in targets_tensors.values()]

# Stack the tensors
X_train = torch.stack(feature_tensorstest)
y_train = torch.stack(target_tensorstest)

print(X_train.shape, y_train.shape)

torch.Size([12, 4, 12992, 224]) torch.Size([12, 25, 3])


In [112]:
# Make torch DataLoader
batch_size = 2

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

In [113]:
# Check DataLoader batches
for i, train_batch in enumerate(train_loader):
    x_train_batch, y_train_batch = train_batch
    print(f'train tensor {i+1}', '|', x_train_batch.shape, '|', y_train_batch.shape)

train tensor 1 | torch.Size([2, 4, 12992, 224]) | torch.Size([2, 25, 3])
train tensor 2 | torch.Size([2, 4, 12992, 224]) | torch.Size([2, 25, 3])
train tensor 3 | torch.Size([2, 4, 12992, 224]) | torch.Size([2, 25, 3])
train tensor 4 | torch.Size([2, 4, 12992, 224]) | torch.Size([2, 25, 3])
train tensor 5 | torch.Size([2, 4, 12992, 224]) | torch.Size([2, 25, 3])
train tensor 6 | torch.Size([2, 4, 12992, 224]) | torch.Size([2, 25, 3])
